# Práctica 5: transformación de datos categóricos (Covid)

## Transformación de variables numéricas
Es convertir una variable categórica en una variable numérica. La transformación de variables categóricas es casi obligatoria para la mayoría de los modelos de aprendizaje de máquinas porque sólo pueden manejar valores numéricos. También se denomina codificación (encoding), o en la minería de textos, la incrustación (embedding). (Tarea encoding y embedding)

Existen muchas técnicas de transformación de datos, entre ellas, hay muchas que utilizan parámetros, como la media y la desviación estándar de la normalización o una tabla de conversión en la codificación de categorías. Un error común en el uso de la transformación variable es transformar el conjunto de entrenamiento y el conjunto de pruebas por separado utilizando parámetros diferentes. La transformación de los datos de entrenamiento y los datos de prueba debe hacerse utilizando los mismos parámetros, y normalmente se obtienen del conjunto de entrenamiento; de lo contrario, no podemos comparar los resultados justamente.

![](tranformacion.png)

Algunos dirán más estrictamente que cuando se trabaje con validación cruzada, los parámetros de transformación se derivarán sólo del volúmen de entrenamiento (k-fold), entonces los datos del volúmen de validación se transformarán por esos parámetros, en lugar de transformar todos los datos de entrenamiento antes de la validación cruzada. Ese enfoque podría ser necesario cuando se prevé que hay enormes diferencias en la distribución de los datos entre los volúmen. La presencia de valores atípicos es una posible razón para dar parámetros diferentes, en particular en las técnicas de transformación sensibles a los valores atípicos, como la escala mín-máx.

In [1]:

conda install -c conda-forge category_encoders

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


## Codificación de un paso (one-hot encoding)
La codificación de un paso es un enfoque para convertir una columna categórica en múltiples columnas binarias (0 o 1), tantas como el número de niveles o categorías distintas de la columna original. Por ejemplo, si hay cuatro niveles en la variable categórica, la codificación de un paso creará cuatro nuevas columnas, cada una de las cuales tiene 0 o 1 y representa si la columna original tiene el nivel.



![](onehot.png)

Una desventaja de la codificación de un paso es que el número de columnas se incrementa fácilmente con muchos niveles distintos. Una posible solución es agrupar algunos niveles sobre la base del conocimiento del dominio o agrupar los niveles poco frecuentes en "otro" nivel.

In [2]:
#Importamos librería y cargamos dataset
import pandas as pd
path = 'covid_mx.csv'
df = pd.read_csv(path, encoding='latin', error_bad_lines=False)
df.head()

,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado


La codificación de un paso se puede hacer la función get_dummies de pandas o con el OneHotEncoder de scikit-learn.

In [3]:
pd.get_dummies(df,columns=['Estado'])

,N° Caso,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real,Estado_AGUASCALIENTES,Estado_BAJA CALIFORNIA,Estado_BAJA CALIFORNIA SUR,Estado_CAMPECHE,Estado_CHIAPAS,...,Estado_QUINTANA ROO,Estado_SAN LUIS POTOSÍ,Estado_SINALOA,Estado_SONORA,Estado_TABASCO,Estado_TAMAULIPAS,Estado_TLAXCALA,Estado_VERACRUZ,Estado_YUCATÁN,Estado_ZACATECAS
0,1,MASCULINO,22,04/04/2020,Confirmado,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,MASCULINO,40,17/03/2020,Confirmado,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,FEMENINO,29,26/03/2020,Confirmado,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,FEMENINO,84,26/03/2020,Confirmado,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,MASCULINO,54,20/03/2020,Confirmado,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,12363,MASCULINO,32,43927,Sospechoso,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12363,12364,FEMENINO,27,43927,Sospechoso,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12364,12365,FEMENINO,41,43920,Sospechoso,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12365,12366,MASCULINO,35,43913,Sospechoso,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Codificación de etiquetas (label encoding)
La codificación de etiquetas es un enfoque para convertir los niveles en enteros. Este enfoque no es apropiado en la mayoría de los algoritmos de aprendizaje automático porque la cantidad de valor transformado en realidad no tiene nada que ver con la variable objetivo, excepto los modelos basados en el árbol de decisión que pueden ser capaces de dividir la columna numérica transformada varias veces con la estratificación del nodo del árbol. Además, en el caso de que la variable categórica tenga una naturaleza "ordinal", por ejemplo, Frío < Calor < Caliente < Muy caliente, la codificación de etiquetas puede funcionar potencialmente mejor que otras técnicas de codificación.

![](labelencoding.png)

Se puede implementar la codificación de etiquetas con la función LabelEncoder de scikit-learn.

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df.head()

,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado


In [5]:
le = LabelEncoder()
le.fit(df['Estado'])
df['Estado_le'] = le.transform(df['Estado'])
df

,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real,Estado_le
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado,27
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado,6
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado,6
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado,16
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado,18
...,...,...,...,...,...,...,...
12362,12363,COAHUILA,MASCULINO,32,43927,Sospechoso,7
12363,12364,CIUDAD DE MÉXICO,FEMENINO,27,43927,Sospechoso,6
12364,12365,NUEVO LEÓN,FEMENINO,41,43920,Sospechoso,18
12365,12366,PUEBLA,MASCULINO,35,43913,Sospechoso,20


## Feature hash
Feature hash es un enfoque para convertir una columna categórica en múltiples columnas usando trucos de hashing. Puede definir el número de nuevas columnas a las que convierte, que puede ser menor que el número de niveles en las columnas categóricas. En lugar de asignar 0 o 1 como una codificación de un solo dígito, el hashing de características utiliza más de dos valores (-1, 0 o 1 en el caso que se indica a continuación).



![](futurehash.png)

Este etiquetado puede cubrir la deficiencia de la codificación de un solo paso que genera demasiadas columnas después de la transformación. Sin embargo, tener demasiadas columnas ya no es una cuestión fatal en las recientes técnicas avanzadas de modelado, por lo que el "hashing" de características no se utiliza ampliamente. Además, tener más de dos valores posibles en una nueva columna puede no ser bueno para algunos modelos.

In [6]:
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
df.head()

,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real,Estado_le
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado,27
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado,6
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado,6
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado,16
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado,18


Se puede utilizar la función FeatureHasher de scikit-learn para realizar este método.

In [7]:
fh = FeatureHasher(n_features=10, input_type='string')
hashed = fh.transform(df[['Estado']].astype(str).values)
hashed = pd.DataFrame(hashed.todense())
hashed

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
12363,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
12364,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
12365,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
hashed.columns = ['Estado_fh' + str(i) for i in hashed.columns]
pd.concat([df, hashed], axis=1)

,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real,Estado_le,Estado_fh0,Estado_fh1,Estado_fh2,Estado_fh3,Estado_fh4,Estado_fh5,Estado_fh6,Estado_fh7,Estado_fh8,Estado_fh9
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado,27,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado,6,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado,6,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado,18,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,12363,COAHUILA,MASCULINO,32,43927,Sospechoso,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
12363,12364,CIUDAD DE MÉXICO,FEMENINO,27,43927,Sospechoso,6,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
12364,12365,NUEVO LEÓN,FEMENINO,41,43920,Sospechoso,18,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
12365,12366,PUEBLA,MASCULINO,35,43913,Sospechoso,20,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Codificación binaria
La codificación binaria es un enfoque para convertir una columna categórica en múltiples columnas binarias, minimizando al mismo tiempo el número de nuevas columnas. En primer lugar, convertir el valor categórico en números enteros en algunos órdenes (por ejemplo, en orden alfabético o en orden de aparición para la fila superior). A continuación, se convierte en un dígito binario de tal manera que 1 a 1, 2 a 10, 5 a 101, etc. Finalmente, se diviode el dígito binario en columnas separadas, cada una de las cuales tiene un solo dígito (1 o 0).

![](binaria.png)

In [9]:
from category_encoders import BinaryEncoder
import pandas as pd
df_x = df
be = BinaryEncoder()
df_be = be.fit_transform(df_x['Estado'])
pd.concat([df_x,df_be],axis=1)


,N° Caso,Estado,Sexo,Edad,Fecha de Inicio de síntomas,Identificación de COVID-19 por RT-PCR\nen tiempo real,Estado_le,Estado_0,Estado_1,Estado_2,Estado_3,Estado_4,Estado_5
0,1,TAMAULIPAS,MASCULINO,22,04/04/2020,Confirmado,27,0,0,0,0,0,1
1,2,CIUDAD DE MÉXICO,MASCULINO,40,17/03/2020,Confirmado,6,0,0,0,0,1,0
2,3,CIUDAD DE MÉXICO,FEMENINO,29,26/03/2020,Confirmado,6,0,0,0,0,1,0
3,4,MÉXICO,FEMENINO,84,26/03/2020,Confirmado,16,0,0,0,0,1,1
4,5,NUEVO LEÓN,MASCULINO,54,20/03/2020,Confirmado,18,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,12363,COAHUILA,MASCULINO,32,43927,Sospechoso,7,0,0,1,1,0,1
12363,12364,CIUDAD DE MÉXICO,FEMENINO,27,43927,Sospechoso,6,0,0,0,0,1,0
12364,12365,NUEVO LEÓN,FEMENINO,41,43920,Sospechoso,18,0,0,0,1,0,0
12365,12366,PUEBLA,MASCULINO,35,43913,Sospechoso,20,0,0,0,1,1,0


## Codificación Base N
La codificación de Base N es la generalización de la codificación binaria en el sentido de que, en lugar de utilizar la base 2, BaseN utiliza un número arbitrario como base. A medida que N aumenta el número de nuevas columnas se reduce, pero también significa que hay más información superpuesta dentro de las nuevas columnas que la que vimos en el caso de la codificación binaria, lo que potencialmente empeora el modelo final. Si N es infinito, la codificación de la BaseN es exactamente la misma que la de la etiqueta. Como ya se ha dicho, la codificación de etiquetas es inapropiada para la mayoría de los modelos.

![](basen.png)

In [10]:
La función BaseNEncoder de la librería category_encoder puede ser útil para este método.

SyntaxError: invalid syntax (<ipython-input-10-2a327b486257>, line 1)

In [ ]:
from category_encoders import BaseNEncoder
import pandas as pd

bne = BaseNEncoder(base=3)
df_bne = bne.fit_transform(df['Estado'])
pd.concat([df, df_bne],axis=1)

## Codificación de frecuencias
La codificación de frecuencias es un enfoque para transformar la columna categórica en una nueva columna con números enteros que representan las frecuencias de los niveles de la columna original. Esto puede funcionar bien cuando la frecuencia de los niveles es influyente para la variable objetivo.

![](frecuencias.png)

También se pueden convertir las frecuencias en rangos, al igual que la serialización de la codificación de frecuencias y la transformación de rangos. Tengan cuidado de que las frecuencias de los niveles tienen más probabilidades de causar empates que la clasificación de los datos originales.

In [ ]:
import pandas as pd
df_train = df
df_test = df_train.iloc[0:2000,]
df_train

No hay ninguna biblioteca (aún) que soporte esta codificación en python pero puede ser fácilmente implementada por la función nativa value_counts() de pandas.

In [ ]:
var='Estado'
df_train=df.copy()
freq = df_train[var].value_counts()
df_train[var+'_fe'] = df_train[var].map(freq)

In [ ]:
df_train

In [ ]:
df_test